# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key
# print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = "../WeatherPy/output_data/weather_df.csv"
weather_data = pd.read_csv(weather_df)
weather_data.count()
weather_data_dpna = weather_data.dropna()
weather_data_dpna.head()
# weather_data_dpna.count()
print(weather_data_dpna.dtypes)

City_name           object
Cloudiness           int64
Country             object
Date                 int64
Humidity             int64
Latitude           float64
Longitude          float64
Max_temperature    float64
Wind_speed         float64
dtype: object


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = weather_data_dpna[["Latitude", "Longitude"]]
humidity = weather_data_dpna["Humidity"].astype(float)


# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
clouds= weather_data_dpna[weather_data_dpna["Cloudiness"] == 0]
clouds
humidity =clouds[clouds["Humidity"] <50]
humidity
max_temp = humidity[humidity["Max_temperature"] <70]
max_temp
wind_sp = max_temp[max_temp["Wind_speed"] < 10] 
wind_sp
# wind_sp.count()
final_df = wind_sp.reset_index(level=None, drop=True)
final_df
# final_df.dtypes

,City_name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max_temperature,Wind_speed
0,Yangi Marg`ilon,0,UZ,1602991661,47,40.43,71.72,57.20,4.70
1,Letlhakane,0,BW,1602991807,47,-21.42,25.58,69.69,7.27
2,Abhā,0,SA,1602991893,34,18.22,42.51,51.80,7.85
3,Torata,0,PE,1602991933,49,-17.08,-70.84,56.01,1.41
4,Zhangye,0,CN,1602991941,30,38.93,100.45,56.88,9.17
5,Karoi,0,ZW,1602991978,45,-16.81,29.69,61.27,4.63
6,Gazojak,0,TM,1602992004,28,41.19,61.40,61.34,5.97
7,Pishin,0,PK,1602992022,23,30.58,67.00,59.63,0.29
8,Aksu,0,CN,1602992029,31,41.12,80.26,59.00,2.66
9,Pahrump,0,US,1602992041,18,36.21,-115.98,66.99,1.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotels=[]

# Loop through narrowed down dataframe to get nearest hotel:
for city in range(len(final_df["City_name"])):

    lat = final_df.loc[city]["Latitude"]
    lng = final_df.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {"location": city_coords,
              "types": "hotel",
              "radius": 5000,
              "key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  
    
    response = requests.get(base_url, params=params)
    response_json = response.json()
# print(json.dumps(response_json, indent=2, sort_keys=True))    
    try:
        hotels.append(response_json["results"][0]["name"])
    except:
        hotels.append("Hotels not found")
        



In [7]:
#adding the Hotel Names column to the data frame:
final_df["Hotel Names"] = hotels        
final_df

,City_name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max_temperature,Wind_speed,Hotel Names
0,Yangi Marg`ilon,0,UZ,1602991661,47,40.43,71.72,57.20,4.70,Fergana
1,Letlhakane,0,BW,1602991807,47,-21.42,25.58,69.69,7.27,Letlhakane
2,Abhā,0,SA,1602991893,34,18.22,42.51,51.80,7.85,Abha
3,Torata,0,PE,1602991933,49,-17.08,-70.84,56.01,1.41,Villa Cuajone
4,Zhangye,0,CN,1602991941,30,38.93,100.45,56.88,9.17,Zhangye
5,Karoi,0,ZW,1602991978,45,-16.81,29.69,61.27,4.63,Karoi
6,Gazojak,0,TM,1602992004,28,41.19,61.40,61.34,5.97,Gazadzhak
7,Pishin,0,PK,1602992022,23,30.58,67.00,59.63,0.29,Pishīn
8,Aksu,0,CN,1602992029,31,41.12,80.26,59.00,2.66,Aksu Prefecture
9,Pahrump,0,US,1602992041,18,36.21,-115.98,66.99,1.99,Pahrump


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Names}</dd>
<dt>City</dt><dd>{City_name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name

hotel_info = [info_box_template.format(**row) for index, row in final_df.iterrows()]
locations = final_df[["Latitude", "Longitude"]]


In [9]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)
# fig = gmaps.figure()
fig.add_layer(marker_layer)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))